In [1]:
"""
Setting 1:
MIN_LIST_LEN = 12
MAX_LIST_LEN = 12
MAX_STEPS = 96
SUCCESS_REWARD = 1.0
STEP_REWARD = -0.3
COMPARISON_ENTROPY_MULTIPLIER = -0.00
NONFINAL_SWAP_REWARD = 0.35
FINAL_SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0
LONGTERM_GAMMA = 0.99
SHORTTERM_GAMMA = 0.7
EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 1000
LR_SCHEDULER_GAMMA = 0.98
NUM_EPISODES = 200000
EPISODES_SAVE = 1000
ckpt_57000_0.9390_44.94.pth


Setting 2:
MIN_LIST_LEN = 12
MAX_LIST_LEN = 12
MAX_STEPS = 96
SUCCESS_REWARD = 1.0
STEP_REWARD = -0.3
COMPARISON_ENTROPY_MULTIPLIER = -0.00
NONFINAL_SWAP_REWARD = 0.35
FINAL_SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0
LONGTERM_GAMMA = 0.99
SHORTTERM_GAMMA = 0.7
EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 4000
LR_SCHEDULER_GAMMA = 0.98
NUM_EPISODES = 200000
EPISODES_SAVE = 1000
ckpt_86000_0.9440_51.48.pth

Setting 3:
MIN_LIST_LEN = 8
MAX_LIST_LEN = 8
MAX_STEPS = 64
CONTEXT_LEN=256
EMBED_DIM=128
NUM_LAYERS=3
SUCCESS_REWARD = 0.29
STEP_REWARD = -0.3
COMPARISON_ENTROPY_MULTIPLIER = -0.00
NONFINAL_SWAP_REWARD = 0.0
FINAL_SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0
LONGTERM_GAMMA = 0.9999
SHORTTERM_GAMMA = 0.7
EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 6000
LR_SCHEDULER_GAMMA = 0.98
NUM_EPISODES = 200000
EPISODES_SAVE = 1000
ckpt_67000_0.9730_20.56_vs_21.42.pth



"""

    

'\nSetting 1:\nMIN_LIST_LEN = 12\nMAX_LIST_LEN = 12\nMAX_STEPS = 96\nSUCCESS_REWARD = 1.0\nSTEP_REWARD = -0.3\nCOMPARISON_ENTROPY_MULTIPLIER = -0.00\nNONFINAL_SWAP_REWARD = 0.35\nFINAL_SWAP_REWARD = 1.0\nINVALID_ACTION_REWARD = -10.0\nLONGTERM_GAMMA = 0.99\nSHORTTERM_GAMMA = 0.7\nEPS_START = 0.5\nEPS_END = 0.05\nEPS_DECAY = 1000\nLR_SCHEDULER_GAMMA = 0.98\nNUM_EPISODES = 200000\nEPISODES_SAVE = 1000\nckpt_57000_0.9390_44.94.pth\n\n\nSetting 2:\nMIN_LIST_LEN = 12\nMAX_LIST_LEN = 12\nMAX_STEPS = 96\nSUCCESS_REWARD = 1.0\nSTEP_REWARD = -0.3\nCOMPARISON_ENTROPY_MULTIPLIER = -0.00\nNONFINAL_SWAP_REWARD = 0.35\nFINAL_SWAP_REWARD = 1.0\nINVALID_ACTION_REWARD = -10.0\nLONGTERM_GAMMA = 0.99\nSHORTTERM_GAMMA = 0.7\nEPS_START = 0.5\nEPS_END = 0.05\nEPS_DECAY = 4000\nLR_SCHEDULER_GAMMA = 0.98\nNUM_EPISODES = 200000\nEPISODES_SAVE = 1000\nckpt_86000_0.9440_51.48.pth\n\nSetting 3:\nMIN_LIST_LEN = 8\nMAX_LIST_LEN = 8\nMAX_STEPS = 64\nCONTEXT_LEN=256\nEMBED_DIM=128\nNUM_LAYERS=3\nSUCCESS_REWARD = 0.29

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import math
import random

MIN_LIST_LEN = 6
MAX_LIST_LEN = 12
MAX_STEPS = 80

CONTEXT_LEN=320
EMBED_DIM=128
NUM_LAYERS=3

SUCCESS_REWARD = 0.29
STEP_REWARD = -0.3
COMPARISON_ENTROPY_MULTIPLIER = -0.00
NONFINAL_SWAP_REWARD = 0.0
FINAL_SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0
LONGTERM_GAMMA = 0.9999
SHORTTERM_GAMMA = 0.7

EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 3000
LR_SCHEDULER_GAMMA = 0.98
NUM_EPISODES = 200000
EPISODES_SAVE = 1000
OUTPUT_DIR = '/home/mcwave/code/autocode/datasets/rl_sort_transformer_easy/list6_12_transformer3_128_gamma07_step80_v3_merge_2_sorted'

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    'less': 2,
    'equal': 3,
    'more': 4,
    '0': 5,
    '1': 6,
    '2': 7,
    '3': 8,
    '4': 9,
    '5': 10,
    '6': 11,
    '7': 12,
    '8': 13,
    '9': 14,
    '10': 15,
    '11': 16,
    '12': 17,
    '13': 18,
    '14': 19,
    '15': 20,
    'len1': 21,
    'len2': 22,
    'len3': 23,
    'len4': 24,
    'len5': 25,
    'len6': 26,
    'len7': 27,
    'len8': 28,
    'len9': 29,
    'len10': 30,
    'len11': 31,
    'len12': 32,
    'len13': 33,
    'len14': 34,
    'len15': 35,
    'len16': 36,
}
inv_vocab = {v: k for k, v in vocab.items()}

def compute_entropy(N, alpha=1):
    K = 2**N
    values = np.arange(K)
    unnormalized_probs = np.exp(-alpha * values)
    Z = unnormalized_probs.sum()
    probs = unnormalized_probs / Z
    return values, -np.log2(probs)

_, int_entropy = compute_entropy(4)

def get_entropy_of_integer(x):
    x = min(15, abs(x))
    return int_entropy[x]

def compute_min_delta_entropy(comparisons):
    # Initialize the result list to store minDelta values
    min_delta = None

    # Iterate through each pair in the comparisons list
    i = len(comparisons) - 1
    xi, yi = comparisons[i]
    if i == 0:
        # For i = 0, use the first case directly
        min_delta = (xi, min(yi, yi - xi), 0)
    else:
        # For i > 0, compute all possible options and select the minimal one
        options = []

        # Simple Entropy
        simple_entropy = (xi, min(yi, yi - xi), 0)
        options.append(simple_entropy)

        # First Delta Entropy
        xi_prev, yi_prev = comparisons[i - 1]
        first_delta_entropy = (xi - xi_prev, yi - yi_prev, 0)
        options.append(first_delta_entropy)

        # Second Delta Entropy (only valid for i > 1)
        if i > 1:
            xi_prev2, yi_prev2 = comparisons[i - 2]
            second_delta_entropy = (
                (xi - xi_prev) - (xi_prev - xi_prev2),
                (yi - yi_prev) - (yi_prev - yi_prev2),
                0,
            )
            options.append(second_delta_entropy)

        # Arbitrary Position Entropy (only valid for i > 1)
        for j in range(i):
            xj, yj = comparisons[j]
            arbitrary_position_entropy = (
                xi - xj,
                yi - yj,
                min(j, i - j),
            )
            options.append(arbitrary_position_entropy)

        # Find the option with the minimal sum
        min_delta = min(options, key=lambda t: sum([get_entropy_of_integer(x) for x in t]))

    entropy = sum([get_entropy_of_integer(x) for x in min_delta])
    if len(comparisons) == 1:
        return 3 * entropy
    else:
        return entropy

def generate_unique_random_list(length, min_value=1, max_value=100):
    """
    Generates a list of unique random integers within a specified range.
    If duplicates are detected, the list is regenerated.
    
    Args:
        length (int): The length of the list to generate.
        min_value (int): The minimum possible value (inclusive).
        max_value (int): The maximum possible value (inclusive).
        
    Returns:
        list: A list of unique random integers.
    """
    if length > (max_value - min_value + 1):
        raise ValueError("Length exceeds the range of unique values available.")
    
    while True:
        random_list = [random.randint(min_value, max_value) for _ in range(length)]
        if len(set(random_list)) == length:
            return random_list

def split_random_pairs(l):
        """
        Splits the sorted list into two lists by randomly assigning each pair of consecutive elements.
        Each pair ((0,1), (2,3), (4,5), ...) is randomly separated into the two lists.
        If the list has an odd number of elements, the last element is randomly assigned to one of the lists.
        :return: A tuple containing two lists with the split elements.
        """
        list1 = []
        list2 = []
        n = len(l)
        i = 0
        while i < n - 1:
            pair = [l[i], l[i + 1]]
            random.shuffle(pair)
            list1.append(pair[0])
            list2.append(pair[1])
            i += 2
        #
        return list1, list2
        
def inplace_merge(arr, start, mid, end):
    """
    In-place merge of arr[start:mid] and arr[mid:end], both sorted.
    Merges so that arr[start:end] is sorted.
    Uses only Compare and Swap operations and counts them.
    """
    compares = 0
    swaps = 0
    
    i = start
    j = mid
    
    # While both subarrays have elements to compare
    while i < j and j < end:
        # Compare element from left half and right half
        compares += 1
        if arr[i] <= arr[j]:
            # If left element is smaller or equal, move on
            i += 1
        else:
            # The element at j is smaller and should be inserted at position i
            # We'll shift elements from i...j-1 by one step to the right
            # and place arr[j] at position i.

            # Save the element to be inserted
            to_insert = arr[j]

            # Move all elements from [i...j-1] one position right
            # This is done by swapping pairs to simulate a rotation.
            k = j
            while k > i:
                # Compare arr[k-1] with to_insert as we "slide" it over.
                # Even though we know arr[k-1] < to_insert might not matter for correctness,
                # we count this as a compare for fairness. We are effectively "checking"
                # positions for insertion.
                compares += 1
                
                # Swap arr[k] and arr[k-1]
                arr[k], arr[k-1] = arr[k-1], arr[k]
                swaps += 1
                k -= 1                    

            # Now arr[i] = to_insert, and we have effectively inserted the element
            arr[i] = to_insert

            # Move pointers:
            i += 1
            j += 1
            if arr == sorted(arr):
                return compares, swaps

    return compares, swaps

def merge_two_sorted_halves(arr):
    """
    Given an array where the first half is sorted and the second half is sorted,
    merge them in-place and count the number of compare and swap operations.
    """
    n = len(arr)
    mid = n // 2
    compares, swaps = inplace_merge(arr, 0, mid, n)
    return compares + swaps
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN // 2, MAX_LIST_LEN // 2) * 2
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.list = generate_unique_random_list(self.length)
        #l1, l2 = split_random_pairs(self.list)
        #self.list = l1 + l2
        self.list = sorted(self.list[0:self.length//2]) + sorted(self.list[self.length//2:])
        self.original_list = list(self.list)
        self.optimal_steps = merge_two_sorted_halves(list(self.list))
        self.sorted_list = sorted(self.list)
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_original_list(self):
        return self.original_list
    
    def get_list_len(self):
        return len(self.list)
    
    def get_optimal_steps(self):
        return self.optimal_steps

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = STEP_REWARD
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = STEP_REWARD * 2
            else:
                response_token = vocab['more']
                reward = STEP_REWARD
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            final_pos_1 = self.sorted_list.index(self.list[index1])
            final_pos_2 = self.sorted_list.index(self.list[index2])
            if final_pos_1 == index2 or final_pos_2 == index1:
                is_swap_final = True
            else:
                is_swap_final = False
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = FINAL_SWAP_REWARD if is_swap_final else NONFINAL_SWAP_REWARD
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -FINAL_SWAP_REWARD
            else:
                reward = STEP_REWARD
            self.indices = None
        else:
            reward = INVALID_ACTION_REWARD
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [3]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=CONTEXT_LEN):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=EMBED_DIM, nhead=8, num_layers=NUM_LAYERS):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [ ]:
def compute_bellman_returns(raw_rewards, gamma):
    bellman_returns = []
    R = 0
    for r in raw_rewards[::-1]:
        R = r + gamma * R
        bellman_returns.insert(0, R)
    return bellman_returns

# Training Loop
def train(verbose=False):
    # Removed torch.autograd.set_detect_anomaly(True)
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=LR_SCHEDULER_GAMMA)
    
    # Optionally, load a checkpoint
    #checkpoint_path = os.path.join(OUTPUT_DIR, "ckpt_199000_1.0000_13.37.pth")
    #load_checkpoint(checkpoint_path, model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    num_successes = 0
    success_step_cnts = []
    optimal_step_cnts = []
    
    for episode in range(NUM_EPISODES):
        t1 = time.time()
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        comparisons = []
        steps = 0
        
        state = 'expect_action'
        done = False
        success = False

        while not done: #and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
                print(comparisons)
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                output = model(input_seq)  # (seq_len, batch_size, vocab_size)
                # Get logits for the last token
                logits = output[-1, 0, :]  # (vocab_size)

                # Check for NaNs in logits
                if torch.isnan(logits).any():
                    print(f"Episode {episode}, NaNs in logits before masking.")
                    break

                # Get valid tokens based on state
                def get_valid_tokens(state):
                    action_tokens = [vocab['Comparison'], vocab['Swap']]
                    index_tokens = [vocab[str(i)] for i in range(env.length)]
                    if state == 'expect_action':
                        return action_tokens
                    elif state == 'expect_index1':
                        return index_tokens[:-1]
                    elif state == 'expect_index2':
                        return [x for x in index_tokens if x > input_tokens[-1]]
                    else:
                        # Handle unexpected states by defaulting to expect_action
                        return action_tokens

                valid_token_ids = get_valid_tokens(state)

                # Ensure valid_token_ids are within the vocab range
                if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                    print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                    break

                # Mask invalid tokens
                mask_value = -1e9  # Use a large negative value instead of -inf
                mask = torch.full_like(logits, mask_value).to(device)
                mask[valid_token_ids] = 0
                masked_logits = logits + mask

                # Sample action. Have some chance to randomly pick a valid action.
                eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
                if random.random() < eps_threshold:
                    masked_logits = masked_logits / 6

                # Check for NaNs in masked_logits
                if torch.isnan(masked_logits).any():
                    print(f"Episode {episode}, NaNs in masked_logits after masking.")
                    break

                # Compute probabilities
                probs = F.softmax(masked_logits, dim=0)

                # Check for NaNs in probs
                if torch.isnan(probs).any():
                    print(f"Episode {episode}, NaNs in probs after softmax.")
                    break

                try:
                    m = torch.distributions.Categorical(probs)
                    action_token = m.sample()
                    log_prob = m.log_prob(action_token)
                except ValueError as e:
                    print(f"Episode {episode}, error in sampling action: {e}")
                    break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        reward = INVALID_ACTION_REWARD
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    steps += 1
                    if done and reward == SUCCESS_REWARD:
                        success = True
                        if episode % 50 == 0:
                            print("Original:", env.get_original_list())
                            print(decode(input_tokens, inv_vocab))
                    if verbose:
                        print("Reward:", reward)
                    state = 'expect_action'
                else:
                    reward = INVALID_ACTION_REWARD
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                comparisons.append((int(inv_vocab[index1_token.item()]), 
                                    int(inv_vocab[index2_token.item()])))
                response_token, reward, done, current_list = env.step(action_tokens)
                steps += 1
                if done and reward == SUCCESS_REWARD:
                    success = True
                else:
                    reward += COMPARISON_ENTROPY_MULTIPLIER * compute_min_delta_entropy(comparisons)
                if verbose:
                    print("Reward:", reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                reward = INVALID_ACTION_REWARD
                done = True

            rewards.append(reward)
        #
        # Post processing of each episode
        success_rewards = [0.0] * len(rewards)
        if success: 
            num_successes += 1
            success_rewards[-1] = SUCCESS_REWARD
            success_step_cnts.append(steps)
            optimal_step_cnts.append(env.get_optimal_steps())
        #
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"

        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns and loss within autocast
        with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
            # Compute returns
            returns1 = compute_bellman_returns(rewards, SHORTTERM_GAMMA)
            returns2 = compute_bellman_returns(success_rewards, LONGTERM_GAMMA)
            returns = torch.tensor(np.array(returns1) + np.array(returns2)).to(device)

            # Check for NaNs in returns
            if torch.isnan(returns).any():
                print(f"Episode {episode}, NaNs in returns.")
                continue

            # Compute loss
            loss = 0
            for log_prob, R in zip(log_probs, returns):
                loss -= log_prob * R

            # Check for NaNs in loss
            if torch.isnan(loss):
                print(f"Episode {episode}, NaN in loss.")
                continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        t2 = time.time()
        if episode % 1 == 0:
            print(f"Episode {episode}, len{env.get_list_len()}, loss:{loss.item():.4f}, {'succeed' if success else 'fail'}, steps:{steps}, opt steps:{env.get_optimal_steps()}, total reward:{sum(rewards):.4f}, {t2-t1} sec")
            
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / episode_cnt
            #assert len(success_step_cnts) == num_successes
            success_rate = num_successes / episode_cnt
            avg_steps = sum(success_step_cnts) / (len(success_step_cnts) + 0.001)
            avg_optimal_steps = sum(optimal_step_cnts) / len(optimal_step_cnts)
            episode_cnt = 0
            total_reward = 0.0
            num_successes = 0
            total_steps = 0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{success_rate:.4f}_{avg_steps:.2f}_vs_{avg_optimal_steps:.2f}.pth")
            #
            # Reduce the lr
            scheduler.step()
            # Optionally, log the learning rate
            current_lr = scheduler.get_last_lr()[0]
            print(f"Learning rate = {current_lr:.6f}")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Episode 0, len12, loss:-59.3486, fail, steps:12, opt steps:34, total reward:-16.4000, 0.3815305233001709 sec
Episode 1, len6, loss:-69.2498, fail, steps:35, opt steps:7, total reward:-23.1000, 0.21028637886047363 sec
Episode 2, len10, loss:-42.7938, fail, steps:3, opt steps:34, total reward:-10.6000, 0.020933866500854492 sec
Episode 3, len8, loss:-61.0553, fail, steps:17, opt steps:24, total reward:-16.2000, 0.09106159210205078 sec
Episode 4, len10, loss:-51.7851, fail, steps:8, opt steps:19, total reward:-12.8000, 0.04406905174255371 sec
Episode 5, len10, loss:-46.4834, fail, steps:3, opt steps:30, total reward:-11.6000, 0.017412662506103516 sec
Episode 6, len6, loss:-8.8487, fail, steps:0, opt steps:3, total reward:-10.0000, 0.003130674362182617 sec
Episode 7, len10, loss:-27.9581, fail, steps:2, opt steps:31, total reward:-9.3000, 0.010460138320922852 sec
Episode 8, len12, loss:-129.5599, fail, steps:56, opt steps:69, total reward:-30.1000, 0.3892827033996582 sec
Episode 9, len12, l

Episode 75, len12, loss:-132.6651, fail, steps:80, opt steps:49, total reward:-26.9000, 0.5142624378204346 sec
Episode 76, len12, loss:-111.7576, fail, steps:80, opt steps:60, total reward:-24.2000, 0.5182344913482666 sec
Episode 77, len6, loss:-78.8509, fail, steps:80, opt steps:15, total reward:-25.8000, 0.5280673503875732 sec
Episode 78, len12, loss:-19.2355, fail, steps:0, opt steps:38, total reward:-10.0000, 0.02114129066467285 sec
Episode 79, len6, loss:-84.0911, fail, steps:80, opt steps:10, total reward:-26.9000, 0.46300721168518066 sec
Episode 80, len6, loss:-49.5115, fail, steps:12, opt steps:14, total reward:-13.0000, 0.0804591178894043 sec
Episode 81, len10, loss:-107.8937, fail, steps:80, opt steps:40, total reward:-23.9000, 0.4694788455963135 sec
Episode 82, len10, loss:-126.8754, fail, steps:80, opt steps:22, total reward:-27.6000, 0.49091362953186035 sec
Episode 83, len12, loss:-90.2695, fail, steps:15, opt steps:57, total reward:-15.9000, 0.10538053512573242 sec
Episod

Episode 150, len10, loss:-110.4729, fail, steps:80, opt steps:47, total reward:-24.1000, 0.5348355770111084 sec
Episode 151, len12, loss:-133.4990, fail, steps:44, opt steps:43, total reward:-23.3000, 0.2986025810241699 sec
Episode 152, len8, loss:-105.1942, fail, steps:80, opt steps:30, total reward:-26.5000, 0.5050051212310791 sec
Episode 153, len10, loss:-107.3640, fail, steps:80, opt steps:45, total reward:-25.4000, 0.5285718441009521 sec
Episode 154, len6, loss:-82.0589, fail, steps:80, opt steps:8, total reward:-26.5000, 0.5219759941101074 sec
Episode 155, len12, loss:-110.3832, fail, steps:80, opt steps:69, total reward:-23.4000, 0.5340633392333984 sec
Episode 156, len8, loss:-105.0339, fail, steps:80, opt steps:19, total reward:-25.8000, 0.5296108722686768 sec
Episode 157, len6, loss:-85.8270, fail, steps:24, opt steps:21, total reward:-18.6000, 0.17052817344665527 sec
Episode 158, len10, loss:-120.7726, fail, steps:80, opt steps:15, total reward:-26.5000, 0.49132537841796875 s

Episode 224, len12, loss:-115.6143, fail, steps:80, opt steps:51, total reward:-25.1000, 0.5317409038543701 sec
Episode 225, len12, loss:-106.4172, fail, steps:80, opt steps:49, total reward:-24.1000, 0.5351014137268066 sec
Episode 226, len10, loss:-115.9714, fail, steps:80, opt steps:41, total reward:-25.1000, 0.5331845283508301 sec
Episode 227, len12, loss:-122.4235, fail, steps:80, opt steps:35, total reward:-25.4000, 0.5392296314239502 sec
Episode 228, len12, loss:-124.3955, fail, steps:80, opt steps:40, total reward:-26.1000, 0.5338590145111084 sec
Episode 229, len10, loss:-89.4874, fail, steps:80, opt steps:55, total reward:-20.8000, 0.5318224430084229 sec
Episode 230, len10, loss:-107.1731, fail, steps:80, opt steps:34, total reward:-24.5000, 0.5270659923553467 sec
Episode 231, len10, loss:-112.0203, fail, steps:80, opt steps:35, total reward:-25.4000, 0.5922696590423584 sec
Episode 232, len8, loss:-102.2744, fail, steps:80, opt steps:6, total reward:-25.8000, 0.5311322212219238

...

Episode 9387, len6, loss:-67.1418, fail, steps:80, opt steps:18, total reward:-24.0000, 0.549415111541748 sec
Episode 9388, len8, loss:-65.1539, fail, steps:80, opt steps:13, total reward:-24.0000, 0.5490615367889404 sec
Episode 9389, len8, loss:-65.2800, fail, steps:80, opt steps:15, total reward:-24.0000, 0.5492618083953857 sec
Episode 9390, len8, loss:-67.3769, fail, steps:80, opt steps:29, total reward:-24.0000, 0.5497896671295166 sec
Episode 9391, len12, loss:-94.2161, fail, steps:80, opt steps:60, total reward:-24.0000, 0.5503518581390381 sec
Episode 9392, len12, loss:-92.5490, fail, steps:80, opt steps:30, total reward:-24.0000, 0.5498349666595459 sec
Episode 9393, len10, loss:-84.8150, fail, steps:80, opt steps:11, total reward:-24.0000, 0.5991272926330566 sec
Episode 9394, len12, loss:-93.6168, fail, steps:80, opt steps:34, total reward:-24.0000, 0.5490455627441406 sec
Episode 9395, len10, loss:-75.1177, fail, steps:80, opt steps:39, total reward:-24.0000, 0.5498042106628418 s

Episode 9461, len6, loss:-62.6229, fail, steps:80, opt steps:20, total reward:-24.0000, 0.5490968227386475 sec
Episode 9462, len6, loss:-66.4396, fail, steps:80, opt steps:16, total reward:-24.0000, 0.5479621887207031 sec
Episode 9463, len8, loss:-79.9018, fail, steps:80, opt steps:20, total reward:-24.0000, 0.5488436222076416 sec
Episode 9464, len12, loss:-98.5797, fail, steps:80, opt steps:14, total reward:-24.0000, 0.5498039722442627 sec
Episode 9465, len8, loss:-80.8389, fail, steps:80, opt steps:21, total reward:-24.0000, 0.5476560592651367 sec
Episode 9466, len12, loss:-96.0468, fail, steps:80, opt steps:43, total reward:-24.0000, 0.5495026111602783 sec
Episode 9467, len6, loss:-72.2680, fail, steps:80, opt steps:8, total reward:-24.0000, 0.6084494590759277 sec
Episode 9468, len6, loss:-66.2114, fail, steps:80, opt steps:20, total reward:-24.0000, 0.5488338470458984 sec
Episode 9469, len8, loss:-77.7566, fail, steps:80, opt steps:20, total reward:-24.0000, 0.5498971939086914 sec


Episode 9535, len6, loss:-69.5729, fail, steps:80, opt steps:3, total reward:-24.0000, 0.5482919216156006 sec
Episode 9536, len8, loss:-80.8893, fail, steps:80, opt steps:21, total reward:-24.0000, 0.5476925373077393 sec
Episode 9537, len8, loss:-84.0956, fail, steps:80, opt steps:18, total reward:-24.0000, 0.5483133792877197 sec
Episode 9538, len12, loss:-98.3269, fail, steps:80, opt steps:27, total reward:-24.0000, 0.6020145416259766 sec
Episode 9539, len12, loss:-98.3498, fail, steps:80, opt steps:45, total reward:-24.0000, 0.549663782119751 sec
Episode 9540, len12, loss:-89.6790, fail, steps:80, opt steps:39, total reward:-24.0000, 0.549778938293457 sec
Episode 9541, len8, loss:-80.8639, fail, steps:80, opt steps:21, total reward:-24.0000, 0.5498337745666504 sec
Episode 9542, len6, loss:-68.5859, fail, steps:80, opt steps:16, total reward:-24.0000, 0.5484983921051025 sec
Episode 9543, len10, loss:-79.8361, fail, steps:80, opt steps:40, total reward:-24.0000, 0.5484788417816162 sec


Episode 9609, len8, loss:-87.3165, fail, steps:80, opt steps:18, total reward:-24.0000, 0.5485682487487793 sec
Episode 9610, len10, loss:-68.0228, fail, steps:80, opt steps:32, total reward:-24.0000, 0.5494711399078369 sec
Episode 9611, len6, loss:-78.3972, fail, steps:80, opt steps:6, total reward:-24.0000, 0.6005358695983887 sec
Episode 9612, len8, loss:-81.7795, fail, steps:80, opt steps:21, total reward:-24.0000, 0.5489494800567627 sec
Episode 9613, len12, loss:-87.2926, fail, steps:80, opt steps:22, total reward:-24.0000, 0.5496606826782227 sec
Episode 9614, len10, loss:-67.9152, fail, steps:80, opt steps:50, total reward:-24.0000, 0.5494701862335205 sec
Episode 9615, len12, loss:-82.5935, fail, steps:80, opt steps:46, total reward:-24.0000, 0.5496838092803955 sec
Episode 9616, len8, loss:-83.9777, fail, steps:80, opt steps:21, total reward:-24.0000, 0.54953932762146 sec
Episode 9617, len12, loss:-80.9382, fail, steps:80, opt steps:43, total reward:-24.0000, 0.5478622913360596 sec

Episode 9683, len8, loss:-80.7658, fail, steps:80, opt steps:16, total reward:-24.0000, 0.5487136840820312 sec
Episode 9684, len10, loss:-42.1427, fail, steps:80, opt steps:26, total reward:-24.7000, 0.542837381362915 sec
Episode 9685, len12, loss:-74.9879, fail, steps:80, opt steps:46, total reward:-24.0000, 0.5474638938903809 sec
Episode 9686, len12, loss:-74.0822, fail, steps:80, opt steps:49, total reward:-24.0000, 0.6040084362030029 sec
Episode 9687, len10, loss:-52.2078, fail, steps:80, opt steps:43, total reward:-24.0000, 0.5476045608520508 sec
Episode 9688, len10, loss:-45.6956, fail, steps:80, opt steps:25, total reward:-24.0000, 0.548677921295166 sec
Episode 9689, len12, loss:-77.2155, fail, steps:80, opt steps:66, total reward:-24.0000, 0.548525333404541 sec
Episode 9690, len8, loss:-79.7834, fail, steps:80, opt steps:26, total reward:-24.0000, 0.5481340885162354 sec
Episode 9691, len12, loss:-66.8403, fail, steps:80, opt steps:29, total reward:-24.0000, 0.5478990077972412 s